In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pytz

# Temperature Data

In [22]:
df_temperature = pd.read_csv("data/72405013743.csv")

In [23]:
df_temperature['Date_Time'] = pd.to_datetime(df_temperature["DATE"])  + pd.Timedelta('12:00:00')
df_temperature['Date_Time'] = df_temperature["Date_Time"].dt.tz_localize('GMT').dt.tz_convert('America/New_York')

In [24]:
df_temperature['Date_t'] = pd.to_datetime(df_temperature['Date_Time']).dt.date

# Interval Data

In [60]:
df_interval = pd.read_csv('data/Courthouse_interval.csv')

In [61]:
df_interval = df_interval.replace({"": np.nan, "-": np.nan, "NaT":np.nan, "NaN": np.nan, "nan": np.nan, "0":np.nan})

In [62]:
df_interval_new = df_interval.melt(id_vars=['Date', 'Account #', 'Meter #'])
df_interval_new.drop(["Account #", "Meter #"], axis=1, inplace=True)

In [63]:
df_interval_new.rename(columns={"variable": "Time", "value": "use_kwh"}, inplace=True)
df_interval_new["use_kwh"] = df_interval_new["use_kwh"].astype(float)

In [64]:
df_interval_new['Date_Time'] = pd.to_datetime(df_interval_new['Date'] + ' ' + df_interval_new['Time'])

In [65]:
df_interval_new = df_interval_new.set_index('Date_Time')
df_interval_new.index = df_interval_new.index.tz_localize('GMT')
df_interval_new.index = df_interval_new.index.tz_convert('America/New_York')

In [66]:
df_interval_new_dailysum = df_interval_new[~df_interval_new["use_kwh"].isna()].groupby(pd.Grouper(freq='D')).sum("use_kwh")

In [67]:
df_interval_new_dailysum.reset_index(inplace=True)

In [68]:
df_interval_new_dailysum['Date_i'] = pd.to_datetime(df_interval_new_dailysum["Date_Time"]).dt.date

In [69]:
df_interval_new_dailysum["Weekday"] = pd.to_datetime(df_interval_new_dailysum['Date_i']).dt.weekday
df_interval_new_dailysum["Weekend"] = pd.to_datetime(df_interval_new_dailysum['Date_i']).dt.weekday >= 5

In [70]:
df_interval_new_dailysum = pd.merge(df_interval_new_dailysum, df_temperature, left_on=['Date_i'], right_on=['Date_t'], how='inner').drop(columns={"Date_t", 'Date_Time_y'})

In [71]:
df_interval_new_dailysum

,Date_Time_x,use_kwh,Date_i,Weekday,Weekend,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,2014-06-30 00:00:00-04:00,920.2,2014-06-30,0,False,72405013743,6/30/14,38.8472,-77.03454,3,...,11.1,999.9,89.1,,69.1,,0.00,G,999.9,0
1,2014-07-01 00:00:00-04:00,8695.2,2014-07-01,1,False,72405013743,7/1/14,38.8472,-77.03454,3,...,15.9,20.0,93.0,,72.0,,0.00,G,999.9,0
2,2014-07-02 00:00:00-04:00,8729.0,2014-07-02,2,False,72405013743,7/2/14,38.8472,-77.03454,3,...,14.0,21.0,99.0,,75.9,,0.00,G,999.9,0
3,2014-07-03 00:00:00-04:00,8445.6,2014-07-03,3,False,72405013743,7/3/14,38.8472,-77.03454,3,...,19.0,33.0,99.0,,75.0,,0.04,G,999.9,10010
4,2014-07-04 00:00:00-04:00,6416.9,2014-07-04,4,False,72405013743,7/4/14,38.8472,-77.03454,3,...,20.0,28.9,91.9,,71.1,,0.52,G,999.9,10010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2021-06-26 00:00:00-04:00,5812.5,2021-06-26,5,True,72405013743,6/26/21,38.8472,-77.03454,3,...,13.0,21.0,84.9,,62.1,,0.14,G,999.9,10000
2554,2021-06-27 00:00:00-04:00,5876.3,2021-06-27,6,True,72405013743,6/27/21,38.8472,-77.03454,3,...,15.9,24.1,89.1,,73.0,,0.00,I,999.9,0
2555,2021-06-28 00:00:00-04:00,6644.3,2021-06-28,0,False,72405013743,6/28/21,38.8472,-77.03454,3,...,12.0,999.9,91.0,,73.9,,0.00,G,999.9,0
2556,2021-06-29 00:00:00-04:00,6646.8,2021-06-29,1,False,72405013743,6/29/21,38.8472,-77.03454,3,...,14.0,18.1,93.0,,75.9,,0.00,G,999.9,0


In [72]:
df_interval_new_dailysum.set_index(['Date_Time_x'], inplace=True)

In [73]:
df_interval_new_dailysum.index.rename('Date_Time', inplace=True)

In [74]:
df_interval_new_dailysum

,use_kwh,Date_i,Weekday,Weekend,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
Date_Time,,,,,,,,,,,,,,,,,,,,,
2014-06-30 00:00:00-04:00,920.2,2014-06-30,0,False,72405013743,6/30/14,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,11.1,999.9,89.1,,69.1,,0.00,G,999.9,0
2014-07-01 00:00:00-04:00,8695.2,2014-07-01,1,False,72405013743,7/1/14,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,15.9,20.0,93.0,,72.0,,0.00,G,999.9,0
2014-07-02 00:00:00-04:00,8729.0,2014-07-02,2,False,72405013743,7/2/14,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,14.0,21.0,99.0,,75.9,,0.00,G,999.9,0
2014-07-03 00:00:00-04:00,8445.6,2014-07-03,3,False,72405013743,7/3/14,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,19.0,33.0,99.0,,75.0,,0.04,G,999.9,10010
2014-07-04 00:00:00-04:00,6416.9,2014-07-04,4,False,72405013743,7/4/14,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,20.0,28.9,91.9,,71.1,,0.52,G,999.9,10010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-26 00:00:00-04:00,5812.5,2021-06-26,5,True,72405013743,6/26/21,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,13.0,21.0,84.9,,62.1,,0.14,G,999.9,10000
2021-06-27 00:00:00-04:00,5876.3,2021-06-27,6,True,72405013743,6/27/21,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,15.9,24.1,89.1,,73.0,,0.00,I,999.9,0
2021-06-28 00:00:00-04:00,6644.3,2021-06-28,0,False,72405013743,6/28/21,38.8472,-77.03454,3,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",...,12.0,999.9,91.0,,73.9,,0.00,G,999.9,0


# Monthly Data Conversion

In [75]:
df_monthly = pd.read_csv("data/Courthouse_Elec.csv")

In [76]:
df_monthly["Bill Begin Date"] = pd.to_datetime(df_monthly["Bill Begin Date"])  + pd.Timedelta('12:00:00 PM')
df_monthly["Bill Begin Date"] = df_monthly["Bill Begin Date"].dt.tz_localize('GMT').dt.tz_convert('America/New_York')
df_monthly["Bill End Date"] = pd.to_datetime(df_monthly["Bill End Date"])  + pd.Timedelta('12:00:00 PM')
df_monthly["Bill End Date"] = df_monthly["Bill End Date"].dt.tz_localize('GMT').dt.tz_convert('America/New_York')

In [77]:
df_monthly["sum_interval_daily_kwh"] = 0
df_monthly["count_interval_daily_kwh"] = 0
df_monthly["sum_interval_daily_weekday_kwh"] = 0
df_monthly["sum_interval_daily_weekend_kwh"] = 0
df_monthly["count_interval_daily_weekday_kwh"] = 0
df_monthly["count_interval_daily_weekend_kwh"] = 0
df_monthly["use_diff"] = 0
df_monthly["use_pct_diff"] = 0
df_monthly["adj_sum_interval_daily_kwh"] = 0
df_monthly["adj_sum_interval_daily_weekday_kwh"] = 0
df_monthly["adj_sum_interval_daily_weekend_kwh"] = 0
df_monthly["oat_avg"] = 0

In [78]:
df_interval_new_dailysum.index

DatetimeIndex(['2014-06-30 00:00:00-04:00', '2014-07-01 00:00:00-04:00',
               '2014-07-02 00:00:00-04:00', '2014-07-03 00:00:00-04:00',
               '2014-07-04 00:00:00-04:00', '2014-07-05 00:00:00-04:00',
               '2014-07-06 00:00:00-04:00', '2014-07-07 00:00:00-04:00',
               '2014-07-08 00:00:00-04:00', '2014-07-09 00:00:00-04:00',
               ...
               '2021-06-21 00:00:00-04:00', '2021-06-22 00:00:00-04:00',
               '2021-06-23 00:00:00-04:00', '2021-06-24 00:00:00-04:00',
               '2021-06-25 00:00:00-04:00', '2021-06-26 00:00:00-04:00',
               '2021-06-27 00:00:00-04:00', '2021-06-28 00:00:00-04:00',
               '2021-06-29 00:00:00-04:00', '2021-06-30 00:00:00-04:00'],
              dtype='datetime64[ns, America/New_York]', name='Date_Time', length=2558, freq=None)

In [81]:
for rname in df_monthly.index:
    row = df_monthly.loc[rname]
    start_date = pd.to_datetime(row["Bill Begin Date"]).tz_convert('America/New_York').date()
    end_date = pd.to_datetime(row["Bill End Date"]).tz_convert('America/New_York').date()
    kwh_sum = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date)].sum()
    kwh_count = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date)].count()
    temp_avg = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date)].TEMP.mean()
    
    df_monthly.loc[rname, "sum_interval_daily_kwh"] = kwh_sum.values[0]
    df_monthly.loc[rname, "count_interval_daily_kwh"] = kwh_count.values[0]
    df_monthly.loc[rname, 'oat_avg'] = temp_avg
       
    kwh_weekday_sum = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == False)].sum()
    kwh_weekend_sum = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == True)].sum()
    kwh_weekday_count = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == False)].count()
    kwh_weekend_count = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == True)].count()

    temp_weekday_avg = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == False)].TEMP.mean()
    temp_weekend_avg = df_interval_new_dailysum.loc[(df_interval_new_dailysum.index.date >= start_date) & (df_interval_new_dailysum.index.date < end_date) & (df_interval_new_dailysum['Weekend'] == True)].TEMP.mean()

    df_monthly.loc[rname, "sum_interval_daily_weekday_kwh"] = kwh_weekday_sum.values[0]
    df_monthly.loc[rname, "sum_interval_daily_weekend_kwh"] = kwh_weekend_sum.values[0]
    df_monthly.loc[rname, "count_interval_daily_weekday_kwh"] = kwh_weekday_count.values[0]
    df_monthly.loc[rname, "count_interval_daily_weekend_kwh"] = kwh_weekend_count.values[0]
    df_monthly.loc[rname, "oat_avg_week`day"] = temp_weekday_avg
    df_monthly.loc[rname, "oat_avg_weekend"] = temp_weekend_avg

In [82]:
df_monthly["use_diff"] = (df_monthly["Use"] - df_monthly["sum_interval_daily_kwh"])
df_monthly["use_pct_diff"] = df_monthly["use_diff"] / (df_monthly["Use"] + df_monthly["sum_interval_daily_kwh"]) / 2

In [83]:
df_monthly["adj_sum_interval_daily_weekday_kwh"] = df_monthly["sum_interval_daily_weekday_kwh"]
df_monthly.loc[df_monthly["use_pct_diff"] >= .05, "adj_sum_interval_daily_weekday_kwh"] = np.nan

df_monthly["adj_sum_interval_daily_weekend_kwh"] = df_monthly["sum_interval_daily_weekend_kwh"]
df_monthly.loc[df_monthly["use_pct_diff"] >= .05, "adj_sum_interval_daily_weekend_kwh"] = np.nan

df_monthly["adj_sum_interval_daily_kwh"] = df_monthly["sum_interval_daily_kwh"]
df_monthly.loc[df_monthly["use_pct_diff"] >= .05, "adj_sum_interval_daily_kwh"] = np.nan


In [84]:
df_monthly.to_csv("Courthouse_monthly_total.csv")